In [15]:
%load_ext autoreload
%autoreload 2

In [16]:
import sys
import logging

from tqdm import tqdm, tqdm_notebook
import numpy as np
import pandas as pd
import h5py
from IPython.display import display

from dvidutils import LabelMapper
from DVIDSparkServices.util import Timer
from DVIDSparkServices.io_util.labelmap_utils import mapping_from_edges
from DVIDSparkServices.graph_comparison import (load_and_normalize_merge_table, normalize_merge_table,
                                                load_supervoxel_sizes, 
                                                compute_comparison_mapping_table, compute_component_table,
                                                compute_split_merge_stats, frequencies_by_size_thresholds,
                                                extract_edges, MERGE_TABLE_DTYPE)

# No need to add a handler -- root logger already has a handler via DVIDSparkServices.__init__
#handler = logging.StreamHandler(sys.stdout)
#logger.addHandler(handler)
logger = logging.getLogger(__name__)

In [3]:
!pwd
print('')
#!ls

/nrs/flyem/bergs/final-agglo-fixsplit



### Load merge graph tables

In [177]:
old_merge_table_path = '../final-agglo/final_20180312_32nm_16nm_all_cbs32_upto10_cb16_upto10_freeze_all.npy'
old_merge_table = load_and_normalize_merge_table(old_merge_table_path)
old_merge_table_df = pd.DataFrame(old_merge_table)
old_edges = extract_edges(old_merge_table)

In [5]:
new_merge_table_path = 'final_20180312_32nm_16nm_all_cbs32_upto10_cb16_upto10_freeze_all_upd0408_v2.npy'
new_merge_table = load_and_normalize_merge_table(new_merge_table_path)
new_merge_table_df = pd.DataFrame(new_merge_table)
new_edges = extract_edges(new_merge_table)

### Supervoxel sizes

In [121]:
sizes_file = '/groups/flyem/data/scratchspace/copyseg-configs/labelmaps/hemibrain/8nm/compute-8nm-extended-fixed-STATS-ONLY-20180402.192015/supervoxel-sizes.h5'
sv_sizes = load_supervoxel_sizes(sizes_file)

INFO [2018-05-01 17:05:48,861] Volume contains 188243164 supervoxels and 22.5 Teravoxels in total


### Comparison mapping tables

In [11]:
sv_table = compute_comparison_mapping_table(old_edges, new_edges, sv_sizes)
component_table = compute_component_table(sv_table)

INFO [2018-04-23 13:18:28,760] Removing duplicate edges...
INFO [2018-04-23 13:19:15,078] Removing duplicate edges took 0:00:46.317220
INFO [2018-04-23 13:19:15,079] Computing intersection...
INFO [2018-04-23 13:19:52,861] Computing intersection took 0:00:37.781029
INFO [2018-04-23 13:19:52,863] Ensuring identical SV sets...
INFO [2018-04-23 13:21:57,102] Ensuring identical SV sets took 0:02:04.238134
INFO [2018-04-23 13:21:57,104] Computing old mapping...
INFO [2018-04-23 13:24:07,360] Computing old mapping took 0:02:10.254389
INFO [2018-04-23 13:24:07,364] Computing new mapping...
INFO [2018-04-23 13:26:15,488] Computing new mapping took 0:02:08.121306
INFO [2018-04-23 13:26:15,500] Computing intersection mapping...
INFO [2018-04-23 13:28:42,355] Computing intersection mapping took 0:02:26.843481
INFO [2018-04-23 13:28:43,644] Appending supervoxel sizes...
INFO [2018-04-23 13:28:54,008] Appending supervoxel sizes took 0:00:10.361507


## Split/Merge stats (all)

In [12]:
affected_components, split_body_stats, merge_body_stats = compute_split_merge_stats(component_table)

### Split frequencies (all)

In [13]:
frequencies_by_size_thresholds(split_body_stats['remaining_voxels'])

,size range,body count
0,>= 1 Gv,14
1,100 Mv - 1 Gv,497
2,10 Mv - 100 Mv,3494
3,1 Mv - 10 Mv,4290
4,100 kv - 1 Mv,3747
5,10 kv - 100 kv,396
6,< 10 kv,0
7,TOTAL,12438


### Merge frequencies (all)

In [14]:
frequencies_by_size_thresholds(merge_body_stats['remaining_voxels'])

,size range,body count
0,>= 1 Gv,35
1,100 Mv - 1 Gv,716
2,10 Mv - 100 Mv,5396
3,1 Mv - 10 Mv,15073
4,100 kv - 1 Mv,24662
5,10 kv - 100 kv,2842
6,< 10 kv,0
7,TOTAL,48724


## Split/Merge stats (reviewed)

### Load reviewed bodies

In [184]:
path = 'hemibrain_all_proofreader_bodies_041818.csv'
reviewed_bodies = pd.read_csv(path, header=None, names=['body', 'reviewer', 'assignment'])['body']
reviewed_bodies = set(map(np.uint64, reviewed_bodies))

In [16]:
# Reviewed bodies are in the 'old' ID space, so computing the set of affected splits is easy
reviewed_split_bodies = set(split_body_stats.query('old_body in @reviewed_bodies').index)

q = '(old_body in @reviewed_split_bodies)'
components_affected_by_reviewed_splits = affected_components.query(q)

_affected_components, reviewed_split_body_stats, _ = \
    compute_split_merge_stats(components_affected_by_reviewed_splits)

# If we selected the component rows correctly, every one of them should have been used.
assert (_affected_components.values == components_affected_by_reviewed_splits.values).all()


# Computing the set of merges that involve reviewed bodies
# requires searching for bodies that include reviewed components.
merged_bodies_with_reviewed_components = set()
merged_components = affected_components.query('new_body in @merge_body_stats.index')
for new_body, body_components in merged_components.groupby('new_body'):
    if reviewed_bodies.intersection(body_components['old_body']):
        merged_bodies_with_reviewed_components.add(new_body)

q = '(new_body in @merged_bodies_with_reviewed_components)'
components_affected_by_reviewed_merges = affected_components.query(q)

_affected_components, _, reviewed_merge_body_stats = \
    compute_split_merge_stats(components_affected_by_reviewed_merges)

# If we selected the component rows correctly, every one of them should have been used.
assert (_affected_components.values == components_affected_by_reviewed_merges.values).all()

In [17]:
affected_new_bodies = np.concatenate((components_affected_by_reviewed_splits['new_body'].values, 
                                      components_affected_by_reviewed_merges['new_body'].values))
affected_reviewed_bodies = pd.unique(affected_new_bodies.flat)
print(f"{len(affected_reviewed_bodies)} bodies were affected")

19494 bodies were affected


In [75]:
len(merged_bodies_with_reviewed_components)

19323

### Split frequencies (reviewed)

In [18]:
frequencies_by_size_thresholds(reviewed_split_body_stats['remaining_voxels'])

,size range,body count
0,>= 1 Gv,13
1,100 Mv - 1 Gv,339
2,10 Mv - 100 Mv,2654
3,1 Mv - 10 Mv,2905
4,100 kv - 1 Mv,2250
5,10 kv - 100 kv,211
6,< 10 kv,0
7,TOTAL,8372


### Merge frequencies (reviewed)

In [19]:
frequencies_by_size_thresholds(reviewed_merge_body_stats['remaining_voxels'])

,size range,body count
0,>= 1 Gv,33
1,100 Mv - 1 Gv,615
2,10 Mv - 100 Mv,3718
3,1 Mv - 10 Mv,6626
4,100 kv - 1 Mv,7418
5,10 kv - 100 kv,913
6,< 10 kv,0
7,TOTAL,19323


## Split/merge stats (UN-reviewed)

In [20]:
# Reviewed bodies are in the 'old' ID space, so computing the set of unaffected splits is easy
unreviewed_split_bodies = set(split_body_stats.query('old_body not in @reviewed_bodies').index)

q = '(old_body in @unreviewed_split_bodies)'
components_affected_by_unreviewed_splits = affected_components.query(q)

_affected_components, unreviewed_split_body_stats, _ = \
    compute_split_merge_stats(components_affected_by_unreviewed_splits)

# If we selected the component rows correctly, every one of them should have been used.
assert (_affected_components.values == components_affected_by_unreviewed_splits.values).all()


# Computing the set of merges that involve unreviewed bodies
# requires searching for bodies that include reviewed components (and taking those that don't).
merged_bodies_with_unreviewed_components = set()
old_bodies_affected_by_merges_of_unreviewed = set()
merged_components = affected_components.query('new_body in @merge_body_stats.index')
for new_body, body_components in merged_components.groupby('new_body'):
    if not reviewed_bodies.intersection(body_components['old_body']):
        old_bodies_affected_by_merges_of_unreviewed = \
            old_bodies_affected_by_merges_of_unreviewed.union(body_components['old_body'].values)
        merged_bodies_with_unreviewed_components.add(new_body)

q = '(new_body in @merged_bodies_with_unreviewed_components)'
components_affected_by_unreviewed_merges = affected_components.query(q)

_affected_components, _, unreviewed_merge_body_stats = \
    compute_split_merge_stats(components_affected_by_unreviewed_merges)

# If we selected the component rows correctly, every one of them should have been used.
assert (_affected_components.values == components_affected_by_unreviewed_merges.values).all()

### Split frequencies (UN-reviewed)

In [21]:
frequencies_by_size_thresholds(unreviewed_split_body_stats['remaining_voxels'])

,size range,body count
0,>= 1 Gv,1
1,100 Mv - 1 Gv,158
2,10 Mv - 100 Mv,840
3,1 Mv - 10 Mv,1385
4,100 kv - 1 Mv,1497
5,10 kv - 100 kv,185
6,< 10 kv,0
7,TOTAL,4066


### Merge frequencies (UN-reviewed)

In [22]:
frequencies_by_size_thresholds(unreviewed_merge_body_stats['remaining_voxels'])

,size range,body count
0,>= 1 Gv,2
1,100 Mv - 1 Gv,101
2,10 Mv - 100 Mv,1678
3,1 Mv - 10 Mv,8447
4,100 kv - 1 Mv,17244
5,10 kv - 100 kv,1929
6,< 10 kv,0
7,TOTAL,29401


# Combine graphs (accepted edges from each)

In [26]:
# We'll disallow merges or splits to any old bodies that were
# reviewed OR that end up in the same merged component as any reviewed body.
frozen_bodies = set(affected_components.query('new_body in @merged_bodies_with_reviewed_components')['old_body'])
frozen_bodies = set(map(np.uint64, frozen_bodies.union(reviewed_bodies)))

In [64]:
len(frozen_bodies)

178064

In [32]:
# Initialize filtered_new_merge_table_df as a copy of new_merge_table_df,
# with appended columns: ['old_body_a', 'new_body_a', 'old_body_b', 'new_body_b']
filtered_new_merge_table_df = new_merge_table_df.merge(sv_table[['old_body', 'new_body']],
                                                       left_on='id_a', right_index=True, copy=False)

filtered_new_merge_table_df = filtered_new_merge_table_df.merge(sv_table[['old_body', 'new_body']],
                                                                left_on='id_b', right_index=True, copy=False,
                                                                suffixes=('_a', '_b'))

In [33]:
# Accept all new edges EXCEPT merging old bodies and either body is supposed to be frozen
q = '(old_body_a == old_body_b) or ((old_body_a not in @frozen_bodies) and (old_body_b not in @frozen_bodies))'
filtered_new_merge_table_df = filtered_new_merge_table_df.query(q)

In [132]:
# Initialize filtered_old_merge_table_df as a copy of old_merge_table_df,
# with appended columns: ['old_body_a', 'new_body_a', 'old_body_b', 'new_body_b']
filtered_old_merge_table_df = old_merge_table_df.merge(sv_table[['old_body', 'new_body']],
                                                       left_on='id_a', right_index=True, copy=False)

filtered_old_merge_table_df = filtered_old_merge_table_df.merge(sv_table[['old_body', 'new_body']],
                                                                left_on='id_b', right_index=True, copy=False,
                                                                suffixes=('_a', '_b'))

In [133]:
# Reject all old edges EXCEPT merges between frozen bodies that are missing in the new agglo.
#q = '(old_body_a in @frozen_bodies) and (new_body_a != new_body_b)'

# Actually, accept all edges involving any frozen bodies.
# This will result in some dupes, which we're dropping anyway, next
q = '(old_body_a in @frozen_bodies)'
filtered_old_merge_table_df = filtered_old_merge_table_df.query(q)

In [134]:
# The original merge tables included duplicates, so there might be duplicates in the filtered versions.
old_dupe_rows = filtered_old_merge_table_df[['id_a', 'id_b']].duplicated(keep='last')
filtered_old_merge_table_df = filtered_old_merge_table_df[~old_dupe_rows]

new_dupe_rows = filtered_new_merge_table_df[['id_a', 'id_b']].duplicated(keep='last')
filtered_new_merge_table_df = filtered_new_merge_table_df[~new_dupe_rows]

In [135]:
# Combine
combined_merge_table_df = pd.concat((filtered_old_merge_table_df, filtered_new_merge_table_df))

# De-dupe
dupe_rows = combined_merge_table_df.duplicated(['id_a', 'id_b'], 'last')
print(f"There were {dupe_rows.sum()} duplicate edges between the old and new agglos")

## Sanity check -- we didn't carry over any edges from 'old' that were already present in 'new'
## EDIT: Not true any more -- I changed the old table filter.
#assert dupe_rows.sum() == 0

There were 9091541 duplicate edges between the old and new agglos


In [139]:
filtered_old_merge_table_df.shape, filtered_new_merge_table_df.shape, combined_merge_table_df.shape

((9143050, 13), (32523315, 13), (41666365, 13))

In [148]:
#display(filtered_old_merge_table_df.query('id_a == 851377905 or id_b == 851377905'))
#display(filtered_new_merge_table_df.query('id_a == 851377905 or id_b == 851377905'))


In [150]:
# Save
merge_table_columns = [name for (name, dtype) in MERGE_TABLE_DTYPE]
combined_merge_table = combined_merge_table_df[merge_table_columns].to_records(index=False)
assert combined_merge_table.dtype == MERGE_TABLE_DTYPE
np.save('combined_merge_table.npy', combined_merge_table)

In [149]:
old_merge_table.shape, new_merge_table.shape, combined_merge_table.shape

((32249777,), (32716517,), (41666365,))

In [151]:
pd.unique(filtered_old_merge_table_df[['old_body_a', 'old_body_b']].values.flat).shape

(85184,)

### Sanity Checks -- We should not see any changes to reviewed bodies

In [143]:
combined_edges = extract_edges(combined_merge_table)
combined_sv_table = compute_comparison_mapping_table(old_edges, combined_edges, sv_sizes)
combined_component_table = compute_component_table(combined_sv_table)

INFO [2018-04-25 18:39:21,215] Removing duplicate edges...
INFO [2018-04-25 18:40:12,593] Removing duplicate edges took 0:00:51.376863
INFO [2018-04-25 18:40:12,595] Computing intersection...
INFO [2018-04-25 18:40:45,891] Computing intersection took 0:00:33.295257
INFO [2018-04-25 18:40:45,892] Ensuring identical SV sets...
INFO [2018-04-25 18:42:45,239] Ensuring identical SV sets took 0:01:59.346559
INFO [2018-04-25 18:42:45,241] Computing old mapping...
INFO [2018-04-25 18:44:36,892] Computing old mapping took 0:01:51.650759
INFO [2018-04-25 18:44:36,894] Computing new mapping...
INFO [2018-04-25 18:46:19,113] Computing new mapping took 0:01:42.218216
INFO [2018-04-25 18:46:19,115] Computing intersection mapping...
INFO [2018-04-25 18:47:51,841] Computing intersection mapping took 0:01:32.724847
INFO [2018-04-25 18:47:53,178] Appending supervoxel sizes...
INFO [2018-04-25 18:48:03,216] Appending supervoxel sizes took 0:00:10.037088


In [144]:
combined_affected_components, combined_split_body_stats, combined_merge_body_stats = \
    compute_split_merge_stats(combined_component_table)

combined_split_body_stats.shape, combined_merge_body_stats.shape

((2996, 4), (29513, 4))

In [145]:
# Reviewed bodies are in the 'old' ID space, so computing the set of affected splits is easy
combined_reviewed_split_bodies = set(combined_split_body_stats.query('old_body in @reviewed_bodies').index)

q = '(old_body in @reviewed_split_bodies)'
combined_components_affected_by_reviewed_splits = combined_affected_components.query(q)

_affected_components, combined_reviewed_split_body_stats, _ = \
    compute_split_merge_stats(combined_components_affected_by_reviewed_splits)

# If we selected the component rows correctly, every one of them should have been used.
assert (_affected_components.values == combined_components_affected_by_reviewed_splits.values).all()


# Computing the set of merges that involve reviewed bodies
# requires searching for bodies that include reviewed components.
combined_merged_bodies_with_reviewed_components = set()
combined_merged_components = combined_affected_components.query('new_body in @combined_merge_body_stats.index')
for new_body, body_components in combined_merged_components.groupby('new_body'):
    if reviewed_bodies.intersection(body_components['old_body']):
        combined_merged_bodies_with_reviewed_components.add(new_body)

q = '(new_body in @combined_merged_bodies_with_reviewed_components)'
combined_components_affected_by_reviewed_merges = combined_affected_components.query(q)

_affected_components, _, combined_reviewed_merge_body_stats = \
    compute_split_merge_stats(combined_components_affected_by_reviewed_merges)

# If we selected the component rows correctly, every one of them should have been used.
assert (_affected_components.values == combined_components_affected_by_reviewed_merges.values).all()

In [146]:
len(combined_merged_bodies_with_reviewed_components)

0

In [147]:
len(combined_reviewed_split_bodies)

0

## Rename phantoms

### Load phantom lists

In [152]:
phantoms = set(pd.read_csv('phantoms.csv', header=None, names=['sv'])['sv'])
large_renames = pd.read_csv('manual-phantom-renames.csv')
large_phantoms = set(large_renames['old'])
assert len(large_phantoms - phantoms) == 0
small_phantoms = phantoms - large_phantoms

In [153]:
patched_merge_table = combined_merge_table.copy()
patched_merge_table_df = pd.DataFrame(patched_merge_table)

In [154]:
small_phantom_rows_view = combined_merge_table_df.query('id_a in @small_phantoms or id_b in @small_phantoms')

### Choose renames for 'small' phantoms

In [155]:
# Choose renames for small phantoms
small_phantom_rows = small_phantom_rows_view.copy()
small_renames = []

iter_count = 0
small_phantom_to_process = list(small_phantoms)
with tqdm_notebook(total=len(small_phantom_to_process)) as progress:
    while small_phantom_to_process and iter_count < 500:
        iter_count += 1
        phantom = small_phantom_to_process.pop(0)
        row_loc = (small_phantom_rows['id_a'] == phantom) | (small_phantom_rows['id_b'] == phantom)
        rows = small_phantom_rows[row_loc]
        rows = rows.sort_values('score')

        # Find the first connection that isn't also a phantom
        other = None
        for row in list(rows.itertuples()):
            if row.id_a not in small_phantoms:
                other = row.id_a
                break
            elif row.id_b not in small_phantoms:
                other = row.id_b
                break

        if other is None:
            small_phantom_to_process.append(phantom)
            #print(f"Moving phantom {phantom} to the end of the list (N={len(small_phantom_to_process)}).")
            continue

        small_renames.append((phantom, other))

        # Replace in the small_phantoms, so it becomes an option for neighboring phantoms.
        small_phantom_rows.iloc[(small_phantom_rows['id_a'] == phantom).values, 0] = other
        small_phantom_rows.iloc[(small_phantom_rows['id_b'] == phantom).values, 1] = other

        progress.update(1)

# Any IDs left at this point were not connected to any 'real' SVs
# Just map them to zero, and we'll discard them later.
print(f"There were {len(small_phantom_to_process)} phantoms that were not attached to any non-phantoms:")
print(small_phantom_to_process)
for phantom in small_phantom_to_process:
    small_renames.append((phantom, 0))

small_renames = pd.DataFrame(small_renames, columns=['old', 'new'])

HBox(children=(IntProgress(value=0, max=306), HTML(value='')))


There were 7 phantoms that were not attached to any non-phantoms:
[1391938980, 1391938982, 2491461668, 2491461671, 2491461672, 608935042, 608935044]


### Apply renames

In [156]:
small_renames.to_csv('small-phantom-rename-table.csv', index=False, header=True)

In [157]:
all_renames = pd.concat((large_renames, small_renames))
assert all_renames.shape == (330, 2)

renames_old = set(all_renames['old'])
renames_new = set(all_renames['new'])

# Select rows to patch
patch_loc = (  patched_merge_table_df['id_a'].isin(renames_old)
             | patched_merge_table_df['id_b'].isin(renames_old) )

patch_rows = patched_merge_table_df[patch_loc].copy()

# Map old->new
patches_a = patch_rows[['id_a']].merge(all_renames, how='left', left_on=['id_a'], right_on=['old'])
patches_b = patch_rows[['id_b']].merge(all_renames, how='left', left_on=['id_b'], right_on=['old'])

patches_a.index = patch_rows.index
patches_b.index = patch_rows.index

patches_a['new'] = patches_a['new'].fillna(patch_rows['id_a'])
patches_b['new'] = patches_b['new'].fillna(patch_rows['id_b'])

patch_rows['id_a'] = patches_a['new'].astype(np.uint64)
patch_rows['id_b'] = patches_b['new'].astype(np.uint64)

patched_merge_table_df[patch_loc] = patch_rows

### Drop zeros

In [159]:
patched_merge_table_df = patched_merge_table_df.query('id_a != 0 and id_b != 0')

In [160]:
# Did we eliminate all phantoms?
assert patched_merge_table_df['id_a'].isin(phantoms).sum() == 0
assert patched_merge_table_df['id_b'].isin(phantoms).sum() == 0

In [161]:
# Normalize
patched_merge_table = patched_merge_table_df[merge_table_columns].to_records(index=False)
patched_merge_table = patched_merge_table.view(MERGE_TABLE_DTYPE)
patched_merge_table = normalize_merge_table(patched_merge_table)
patched_merge_table_df = pd.DataFrame(patched_merge_table)

# Save
np.save('patched_combined_merge_table.npy', patched_merge_table)

## Explicit merges (to fix errors after splits were applied)

In [163]:
continuity_fixes_edges = pd.read_csv('aux_merges_split_bodies_continuity.csv', header=None, names=['id_a', 'id_b'], dtype=np.uint64)
continuity_fixes_edges = np.sort(continuity_fixes_edges.values, axis=1)

continuity_fixes_merge_table = np.zeros((len(continuity_fixes_edges,)), dtype=MERGE_TABLE_DTYPE)
continuity_fixes_merge_table['id_a'] = continuity_fixes_edges[:,0]
continuity_fixes_merge_table['id_b'] = continuity_fixes_edges[:,1]

In [ ]:
final_merge_table = np.concatenate((patched_merge_table, continuity_fixes_merge_table))
np.save('final_merge_table.npy', final_merge_table)

In [177]:
np.save('../final-agglo-fixsplit-patched/final_patched_20180426_merge_table.npy', final_merge_table)

# Final comparison

In [178]:
# Reload from disk (for starting from fresh kernel)
final_merge_table = np.load('../final-agglo-fixsplit-patched/final_patched_20180426_merge_table.npy')
final_edges = extract_edges(final_merge_table)

(32575002,)

In [179]:
final_sv_table = compute_comparison_mapping_table(old_edges, final_edges, sv_sizes)
final_component_table = compute_component_table(final_sv_table)

INFO [2018-05-02 17:14:02,220] Removing duplicate edges...
INFO [2018-05-02 17:14:52,119] Removing duplicate edges took 0:00:49.897857
INFO [2018-05-02 17:14:52,121] Computing intersection...
INFO [2018-05-02 17:15:27,680] Computing intersection took 0:00:35.558661
INFO [2018-05-02 17:15:27,682] Ensuring identical SV sets...
INFO [2018-05-02 17:19:39,852] Computing old mapping took 0:02:01.728812
INFO [2018-05-02 17:19:39,854] Computing new mapping...
INFO [2018-05-02 17:21:19,389] Computing new mapping took 0:01:39.534640
INFO [2018-05-02 17:21:19,391] Computing intersection mapping...
INFO [2018-05-02 17:22:55,626] Computing intersection mapping took 0:01:36.234430
INFO [2018-05-02 17:22:56,970] Appending supervoxel sizes...
INFO [2018-05-02 17:23:06,884] Appending supervoxel sizes took 0:00:09.913777


In [180]:
final_affected_components, final_split_body_stats, final_merge_body_stats = \
    compute_split_merge_stats(final_component_table)

In [185]:
# Reviewed bodies are in the 'old' ID space, so computing the set of affected splits is easy
final_reviewed_split_bodies = set(final_split_body_stats.query('old_body in @reviewed_bodies').index)

q = '(old_body in @final_reviewed_split_bodies)'
final_components_affected_by_reviewed_splits = final_affected_components.query(q)

_affected_components, final_reviewed_split_body_stats, _ = \
    compute_split_merge_stats(final_components_affected_by_reviewed_splits)

# If we selected the component rows correctly, every one of them should have been used.
assert (_affected_components.values == final_components_affected_by_reviewed_splits.values).all()


# Computing the set of merges that involve reviewed bodies
# requires searching for bodies that include reviewed components.
final_merged_bodies_with_reviewed_components = set()
final_merged_components = final_affected_components.query('new_body in @final_merge_body_stats.index')
for new_body, body_components in final_merged_components.groupby('new_body'):
    if reviewed_bodies.intersection(body_components['old_body']):
        final_merged_bodies_with_reviewed_components.add(new_body)

q = '(new_body in @final_merged_bodies_with_reviewed_components)'
final_components_affected_by_reviewed_merges = final_affected_components.query(q)

_affected_components, _, final_reviewed_merge_body_stats = \
    compute_split_merge_stats(final_components_affected_by_reviewed_merges)

# If we selected the component rows correctly, every one of them should have been used.
assert (_affected_components.values == final_components_affected_by_reviewed_merges.values).all()

In [186]:
final_old_bodies_in_merges = pd.unique(final_components_affected_by_reviewed_merges['old_body'])
len(final_old_bodies_in_merges)

222

In [190]:
len(final_reviewed_split_bodies)

91

In [192]:
pd.Series(final_old_bodies_in_merges).to_csv('old-reviewed-bodies-in-new-merges.csv', index=False, header=False)
final_components_affected_by_reviewed_merges['new_body'].drop_duplicates()\
    .to_csv('new-merged-bodies-to-review.csv', index=False)

In [194]:
!ls $(pwd)/old-reviewed-bodies-in-new-merges.csv
!ls $(pwd)/new-merged-bodies-to-review.csv


/nrs/flyem/bergs/final-agglo-fixsplit/old-reviewed-bodies-in-new-merges.csv
/nrs/flyem/bergs/final-agglo-fixsplit/new-merged-bodies-to-review.csv


In [196]:
final_old_bodies_in_merges.shape, final_components_affected_by_reviewed_merges['new_body'].drop_duplicates().shape

((222,), (81,))

In [199]:
(661403761 in final_old_bodies_in_merges,
661403761 in final_components_affected_by_reviewed_merges['new_body'].values)

(True, False)

In [199]:
# There should be no real 'splits' on old bodies EXCEPT for bodies involving phantoms.
# (The 'phantom' was removed from the final agglo, so it appears 'split' from the body.)
final_split_sv_table = final_sv_table.query('old_body in @final_reviewed_split_bodies')

d = {}
for old_body, table in final_split_sv_table.groupby('old_body'):
    d[old_body] = len(set(table.index).intersection(phantoms))

assert all(d.values()), "An old body was split despite containing no phantoms"

In [179]:
%time final_mapping = mapping_from_edges(final_edges, as_series=True)

CPU times: user 1min 27s, sys: 26.4 s, total: 1min 53s
Wall time: 1min 53s


In [202]:
final_components_affected_by_reviewed_merges['new_body'].to_csv('new-merged-bodies-to-review.csv', index=False)

In [176]:
#!head new-merged-bodies-to-review.csv
mnbtr = pd.read_csv('new-merged-bodies-to-review.csv', header=None, names=['body'])
mnbtr.shape

(247, 1)

In [181]:
len(final_mapping)

41037730

In [182]:
len(sv_table)

41132595

In [324]:
len(final_split_body_stats), len(final_merge_body_stats)

(3279, 29669)

In [327]:
large_merge_body_stats = final_merge_body_stats.query('body_voxels > 10e6')

In [331]:
large_split_body_stats = final_split_body_stats.query('body_voxels > 10e6')
large_split_body_stats.shape

(2969, 4)

In [184]:
pd.DataFrame(final_mapping).query('sv == body').shape

(9655243, 1)

In [185]:
final_mapping_no_identities = pd.DataFrame(final_mapping).query('sv != body')

In [188]:
name = 'final_patched_20180426_mapping_no_identities.csv'
final_mapping_no_identities.to_csv(f'../final-agglo-fixsplit-patched/{name}', index=True)

In [189]:
name = 'final_patched_20180426_mapping.csv'
final_mapping.to_csv(f'../final-agglo-fixsplit-patched/{name}', index=True)

In [205]:
reviewed_svs = sv_table.query('old_body in @reviewed_bodies')

In [206]:
reviewed_merge_table_df = old_merge_table_df.query('id_a in @reviewed_svs.index or id_b in @reviewed_svs.index')

In [212]:
f = final_merge_table_mapped_df.copy()

In [213]:
f.reset_index(inplace=True, drop=True)

In [214]:
f[:5]

,id_a,id_b,score,body
0,106979579,2386468249,5.117693,106979579
1,108002724,108343758,9.293223,108002724
2,108002724,108343809,12.983173,108002724
3,108002724,139378772,4.019378,108002724
4,108002724,139378775,9.128134,108002724


In [215]:
%time f2 = pd.concat((f, f[0:2]), ignore_index=True, copy=False)

CPU times: user 1.01 s, sys: 2.75 s, total: 3.76 s
Wall time: 3.76 s


In [ ]:
import sqlite3
conn = sqlite3.connect('/tmp/final.sqlite')
c = conn.cursor()
#c.execute('CREATE TABLE supervoxels (supervoxel_id INTEGER, agglo_id INTEGER)')
#c.execute('CREATE INDEX supervoxels_by_supervoxel_id_index ON supervoxels (supervoxel_id)')
#c.execute('CREATE INDEX supervoxels_by_agglo_id_index ON supervoxels (agglo_id)')
#c.execute('CREATE TABLE edges (id_a INTEGER, id_b INTEGER, score REAL, body_id INTEGER)')
#c.execute('CREATE INDEX edge_body_index ON edges (body_id)')

f.to_sql('merge_table', )


In [218]:
old_body_sizes['accumulated_voxels'] = np.add.accumulate(old_body_sizes['voxel_count'])

In [259]:
old_body_sizes['accumulated_percent'] = old_body_sizes['accumulated_voxels'] / neuron_total_voxels

In [260]:
(old_body_sizes['accumulated_percent'] < 0.7).sum()

97565

In [227]:
old_body_sizes[248451:248461]

,voxel_count,segment_count,accumulated_voxels,accumulated_percent
body,,,,
1498414905,6332043,4,15723386341095,0.700000
1840872780,6331964,4,15723392673059,0.700000
1694781425,6331949,6,15723399005008,0.700001
1195308802,6331909,2,15723405336917,0.700001
1018247616,6331909,4,15723411668826,0.700001
1377132441,6331877,5,15723418000703,0.700002
2089380989,6331866,8,15723424332569,0.700002
387179983,6331863,7,15723430664432,0.700002
1752175862,6331829,2,15723436996261,0.700002


In [228]:
old_body_sizes['accumulated_percent_of_bodies'] = old_body_sizes['accumulated_voxels'] / old_body_sizes['voxel_count'].sum()

In [229]:
(old_body_sizes['accumulated_percent_of_bodies'] < 0.7).sum()

56788

(188243164,)

In [232]:
sv_sizes.sum() / 1e12

22.461973722635999

In [ ]:
sv_classes = pd.read_csv('classifications.csv')
sv_classes.columns = ['sv', 'class']
sv_classes.set_index('sv', inplace=True)

In [247]:
sv_classes_with_size = pd.DataFrame(sv_sizes).merge(sv_classes, 'left', left_index=True, right_index=True)

In [249]:
sv_classes_with_size.columns = ['voxel_count', 'klass']
sv_classes_with_size.index = sv_classes_with_size.index.values.astype(np.uint64)

In [289]:
neuron_classes = set([5,6])  # Without cell bodies
#neuron_classes = set([3,5,6]) # With cell bodies
neuron_sv_classes_with_size = sv_classes_with_size.query('klass in @neuron_classes')
neuron_total_voxels = neuron_sv_classes_with_size['voxel_count'].sum()

In [290]:
table = []
for size in [0, 1e5, 1e6, 1e7, 1e8]:
    objects = neuron_sv_classes_with_size[neuron_sv_classes_with_size['voxel_count'] > size]
    num_objects = objects.shape[0]
    objects_total_voxels = objects['voxel_count'].sum()
    fraction_of_volume = objects_total_voxels / neuron_total_voxels
    table.append( (size, num_objects, objects_total_voxels, fraction_of_volume*100) )

In [291]:
pd.DataFrame(table, columns=['size', 'num objects', 'total voxels', 'fraction of total voxels'])

,size,num objects,total voxels,fraction of total voxels
0,0.0,147925884,16454639639102,100.000000
1,100000.0,7351147,14657389844439,89.077550
2,1000000.0,2451815,13192847370777,80.177067
3,10000000.0,165584,6787770183114,41.251406
4,100000000.0,11585,3114323515944,18.926720


In [292]:
#neuron_classes = set([5,6])  # Without cell bodies
neuron_classes = set([3,5,6]) # With cell bodies
neuron_sv_classes_with_size = sv_classes_with_size.query('klass in @neuron_classes')
neuron_total_voxels = neuron_sv_classes_with_size['voxel_count'].sum()

In [293]:
table = []
for size in [0, 1e5, 1e6, 1e7, 1e8]:
    objects = neuron_sv_classes_with_size[neuron_sv_classes_with_size['voxel_count'] > size]
    num_objects = objects.shape[0]
    objects_total_voxels = objects['voxel_count'].sum()
    fraction_of_volume = objects_total_voxels / neuron_total_voxels
    table.append( (size, num_objects, objects_total_voxels, fraction_of_volume*100) )

In [294]:
pd.DataFrame(table, columns=['size', 'num objects', 'total voxels', 'fraction of total voxels'])

,size,num objects,total voxels,fraction of total voxels
0,0.0,159903575,20029685679301,100.000000
1,100000.0,8330175,18133561147779,90.533428
2,1000000.0,2688608,16439615151039,82.076251
3,10000000.0,217074,9500789425405,47.433542
4,100000000.0,16825,4176162089339,20.849863


In [300]:
type(sv_classes)

pandas.core.frame.DataFrame

In [320]:
old_mapper = LabelMapper(old_mapping.index.values.astype(np.uint64), old_mapping.values[:, 0].astype(np.uint64))

In [321]:
sv_classes_with_size['old_body'] = old_mapper.apply(sv_classes_with_size.index.values, allow_unmapped=True)

In [323]:
sv_classes_with_size[:10]

,voxel_count,klass,old_body
79355095,335838088,1.0,79355095
79359413,195016,7.0,79359413
79359414,9616,7.0,79359414
79696138,75229447,1.0,79696138
79696139,480,1.0,79696139
79696140,339152,1.0,79696140
79696160,80,1.0,79696160
79696161,2,NaN,79696161
79700460,80,1.0,79700460
79700468,8984,7.0,79700468


In [325]:
body_classes_with_sizes = sv_classes_with_size.groupby('old_body').agg({'klass': 'min', 'voxel_count': 'sum'})

In [333]:
#neuron_classes = set([5,6])  # Without cell bodies
neuron_classes = set([3,5,6]) # With cell bodies
neuron_body_classes_with_size = body_classes_with_sizes.query('klass in @neuron_classes')
neuron_total_voxels = neuron_body_classes_with_size['voxel_count'].sum()

table = []
for size in [0, 1e5, 1e6, 1e7, 1e8]:
    objects = neuron_body_classes_with_size[neuron_body_classes_with_size['voxel_count'] > size]
    num_objects = objects.shape[0]
    objects_total_voxels = objects['voxel_count'].sum()
    fraction_of_volume = objects_total_voxels / neuron_total_voxels
    table.append( (size, num_objects, objects_total_voxels, fraction_of_volume*100) )

pd.DataFrame(table, columns=['size', 'num objects', 'total voxels', 'fraction of total voxels'])

In [337]:
neuron_body_classes_with_size.sort_values('voxel_count', ascending=False, inplace=True)

/groups/flyem/proj/cluster/miniforge/envs/flyem/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [340]:
neuron_body_classes_with_size['accumulated_voxel_count'] = \
    np.add.accumulate(neuron_body_classes_with_size['voxel_count'])

/groups/flyem/proj/cluster/miniforge/envs/flyem/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [351]:
index_70pct = (neuron_body_classes_with_size['accumulated_voxel_count'] / neuron_body_classes_with_size['voxel_count'].sum() > 0.7).values.nonzero()[0][0]
print(index_70pct)
neuron_body_classes_with_size[index_70pct:index_70pct+1]

191572


,klass,voxel_count,accumulated_voxel_count
old_body,,,
541620506,6.0,7356734.0,9.748614e+12


In [345]:
neuron_classes = set([5,6])  # Without cell bodies
#neuron_classes = set([3,5,6]) # With cell bodies
neuron_body_classes_with_size = body_classes_with_sizes.query('klass in @neuron_classes')
neuron_total_voxels = neuron_body_classes_with_size['voxel_count'].sum()

table = []
for size in [0, 1e5, 1e6, 1e7, 1e8]:
    objects = neuron_body_classes_with_size[neuron_body_classes_with_size['voxel_count'] > size]
    num_objects = objects.shape[0]
    objects_total_voxels = objects['voxel_count'].sum()
    fraction_of_volume = objects_total_voxels / neuron_total_voxels
    table.append( (size, num_objects, objects_total_voxels, fraction_of_volume*100) )

pd.DataFrame(table, columns=['size', 'num objects', 'total voxels', 'fraction of total voxels'])

,size,num objects,total voxels,fraction of total voxels
0,0.0,118085483,1.392659e+13,100.000000
1,100000.0,4696658,1.257657e+13,90.306132
2,1000000.0,891795,1.163720e+13,83.561018
3,10000000.0,144620,9.347256e+12,67.118047
4,100000000.0,19889,6.077680e+12,43.640831


In [347]:
neuron_body_classes_with_size.sort_values('voxel_count', ascending=False, inplace=True)

/groups/flyem/proj/cluster/miniforge/envs/flyem/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [350]:
neuron_body_classes_with_size['accumulated_voxel_count'] = \
    np.add.accumulate(neuron_body_classes_with_size['voxel_count'])
index_70pct = (neuron_body_classes_with_size['accumulated_voxel_count'] / neuron_body_classes_with_size['voxel_count'].sum() > 0.7).values.nonzero()[0][0]
print(index_70pct)
neuron_body_classes_with_size[index_70pct:index_70pct+1]

/groups/flyem/proj/cluster/miniforge/envs/flyem/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


191572


,klass,voxel_count,accumulated_voxel_count
old_body,,,
541620506,6.0,7356734.0,9.748614e+12
